In [1]:
import zCurve as z
import csv
import os
import numpy as np
import pandas as pd
import glob
import json 
import matplotlib.pyplot as plt

In [66]:
def calculateMortonFromList_with_zCurve(values):
    # Cap floating point numbers to one decimal place and convert to integers
    int_values = [int(round(value, 1) * 10) for value in values]
    value = z.interlace(*int_values, dims=len(int_values))
    return value

In [67]:
roi_mean_values = pd.read_csv('../outputs/smirk_roi_mean_values.csv')
roi_mean_values

,frame,cell_1,cell_2,cell_3,cell_4,cell_5,cell_6
0,child/Y4kC5HWcultxGWAhuKdZw/cam000006.png,0.002120,0.028477,4.688508,0.375582,0.316727,0.124537
1,child/XpIss6USqq1CvIRb2weqE/cam000280.png,0.002110,0.000000,2.135890,0.193213,0.324183,0.152763
2,child/BLt5th4KUYDe4tqqwQEbN/cam000128.png,0.000000,0.000000,3.505804,0.553993,0.531435,0.161224
3,child/1zJO0PfH4rKy8MOl5x5di/cam000035.png,0.000699,0.000000,0.040380,28.866972,11.626774,0.130466
4,child/0i0Pv5GElXKQI1OoIHfbY/cam000025.png,8.619019,30.753439,0.462520,0.417660,0.377722,0.113543
...,...,...,...,...,...,...,...
69296,child/FUTcU9qixnZnRtUKfR73U/cam000022.png,0.000000,0.000000,0.891905,2.648956,0.357865,0.112293
69297,child/Hg5IVmPBKY2WzuemE7p6J/cam000033.png,0.000320,0.000000,0.100777,2.350223,0.330376,0.116829
69298,child/4C874IC44CDuep4rdQFIN/cam000171.png,0.000283,0.000000,0.047996,29.854540,1.106992,0.164594
69299,child/kJEr4oDRPCYEqv6dQgu6o/cam000043.png,0.000000,0.000000,33.496441,0.542867,0.287053,0.116550


In [68]:
cell_columns = [col for col in roi_mean_values.columns if col.startswith("cell")]
cell_columns

['cell_1', 'cell_2', 'cell_3', 'cell_4', 'cell_5', 'cell_6']

In [69]:
for _, row in roi_mean_values.iterrows():
    print('frame_roi: \n', j)
    value = calculateMortonFromList_with_zCurve(row[col] for col in cell_columns)
    print('morton: \n ', value)
    break

frame_roi: 
 frame     child/Y4kC5HWcultxGWAhuKdZw/cam000006.png
cell_1                                      0.00212
cell_2                                     0.028477
cell_3                                     4.688508
cell_4                                     0.375582
cell_5                                     0.316727
cell_6                                     0.124537
Name: 0, dtype: object
morton: 
  4296066356


In [75]:
cell_columns = [col for col in roi_mean_values.columns if col.startswith("cell")]
morton_codes = []
i=0
for _, row in roi_mean_values.iterrows():
    value = calculateMortonFromList_with_zCurve(row[col] for col in cell_columns)
    frame = row['frame']
    morton_codes.append({'frame': frame, 'morton': value})
#     i+=1
#     if i > 30:
#         break
# print(morton_codes)
morton_codes = pd.DataFrame(morton_codes, columns=['frame', 'morton'])
morton_codes.to_csv('../outputs/morton_codes_all.csv', sep=';', index=False)